In [ ]:
import os
import weaviate
from wcs_key import wcs_token

auth_config = weaviate.auth.AuthApiKey(api_key=wcs_token)  # Replace w/ your API Key for the Weaviate instance

client = weaviate.Client(
    url="https://workshop-prep-cluster-v54dbajt.weaviate.network",
    auth_client_secret=auth_config,  #weaivate instance API key
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),   # Replace w/ your Cohere Key 
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),   # Replace w/ your OpenAI Key 
    }
)
client.is_ready()

## Search through your data
1. Classic Word Search
2. Vector Search

In [ ]:
def word_search(word):
    where_filter = {
      "path": ["title"],
      "operator": "Like", #
      "valueString": word,
    }

    query_result = (
      client.query
      .get("Article", ["title", "text"])
      .with_where(where_filter)
      .do()
    )
    
    return query_result['data']['Get']['Article'][0]['text']

print(word_search("Avocado"))

In [ ]:
word_search("Data science")

In [ ]:
word_search("fast animals")

In [ ]:
where_filter = {
      "path": ["title"],
      "operator": "Like", #
      "valueString": 'fast animal',
    }

query_result = (
      client.query
      .get("Article", ["title", "text"])
      .with_where(where_filter)
      .do()
    )

query_result

## Semantic(Vector) Search

In [ ]:
def semantic_search(query):
    nearText = {
        "concepts": [query],
    }

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("Article", properties)
        .with_near_text(nearText)
        .with_limit(3)
        .do()
    )

    result = response['data']['Get']['Article']

    return result
    
def print_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

In [ ]:
query_result = semantic_search("a programming language used for machine learning")

print_result(query_result)

In [ ]:
query_result = semantic_search("fast animals")

print_result(query_result)

## This is a Multi-Lingual Model!!

In [ ]:
#great movies in chinese

query_result = semantic_search("很棒的电影")

print_result(query_result)

In [ ]:
#great movies in hindi

query_result = semantic_search("महान फिल्में")

print_result(query_result)

In [ ]:
#vacation spots in Farsi

query_result = semantic_search("مکان های تعطیلات")

print_result(query_result)